In [1]:
print("ok")

ok


In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\AI\\GenAI\\my_projects\\medical_chatbot_genAI'

In [4]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

d:\AI\GenAI\my_projects\medical_chatbot_genAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#Extract data from pdf files in the directory

def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="**/*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [6]:
extract_data=load_pdf_file("Data/")

In [7]:
def text_splitter(extract_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extract_data)
    return text_chunks

In [8]:
text_chunks=text_splitter(extract_data)
print("length of text chunks:",len(text_chunks))

length of text chunks: 5860


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

#download the embdddings from hugging face

def download_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings=download_embeddings()

C:\Users\USER\AppData\Local\Temp\ipykernel_20268\2377785644.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [11]:
query_test=embeddings.embed_query("What is the purpose of the trial?")
print("length of query embedding:",len(query_test))

length of query embedding: 384


In [12]:
#query_test

In [25]:
from pinecone import ServerlessSpec
from pinecone.grpc import PineconeGRPC as pinecone
import os
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")

In [14]:
pc=pinecone(api_key=PINECONE_API_KEY)

index_name="medicalbot"

# Check if index exists before creating
if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name,
                    dimension=384,
                    metric="cosine",
                    spec=ServerlessSpec(cloud="aws",
                                        region="us-east-1"))
    print(f"Index '{index_name}' created. Waiting for it to be ready...")
    
    # Wait for the index to be ready
    import time
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
    print(f"Index '{index_name}' is now ready!")
else:
    print(f"Index '{index_name}' already exists")

Index 'medicalbot' already exists


In [26]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
OpenAI_API_KEY=os.environ.get("OPENAI_API_KEY")

In [16]:
#embed each chunk and upsert the embeddings into your pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [17]:
#load Existing index

from langchain_pinecone import PineconeVectorStore
#embed each chink and upsert the embeddings into your pinecone index
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [18]:
retriver=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [19]:
retriver_docs=retriver.invoke("what is acne?")

In [20]:
retriver_docs

[Document(id='fd9a95c0-74a5-465c-b7e6-958aab350e5a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='055e8911-ab47-478e-b748-4fe6b22b698d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='23699b78-18fc-4984-832d-02c09ca8faf4', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38, 'page_labe

In [27]:
from langchain_openai import OpenAI
llm=OpenAI(temperature=0.4,max_tokens=500)

In [32]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [33]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriver,question_answer_chain)

In [36]:
response=rag_chain.invoke({"input": "what is acne?"})
print(response["answer"])



Acne is a skin disorder characterized by inflammation of the sebaceous glands. It is commonly referred to as pimples or zits. It can affect both men and women and typically appears on the face, but can also occur on other parts of the body.
